what did I do? The clean code for webscrapping via API with CPV code. 

1-KEYWORDS SCRAP

In [112]:
scraped_data = []
response = requests.get('https://prod.simap.ch/api/publications/v2/project/project-search', params={'cpvCodes': ['72000000', '79300000']})
response_dict = response.json()
scraped_data.extend([{field: project[field] for field in required_fields} for project in response_dict["projects"]])



In [ ]:
keywords = [
    "study","studie","étude","etude","studio",
    "analysis","analyse","analisi",
    "economy","economics","wirtschaft","ökonomie","économie","economie","economia",
    "benchmark","benchmarking",
    "wirtschaftsberatung","consulting","conseil","consulenza",
    "it-dienste","dienstleistung","service","prestation","prestazione",
    "ausschreibung","appel d'offres","gara","procurement",
    "offenes verfahren","procédure ouverte","procedura aperta",
    "bundesamt für statistik","office fédéral de la statistique","bfs","seco",
    "kanton zürich","kanton luzern","region","bundesamt für bauten und logistik","bbl",
    "index","wirtschaftsforschung","tunnel"
    
]


In [ ]:
import requests
import pandas as pd
from time import sleep

url = "https://archiv.simap.ch/api/search?pageNo=1&recordsPerPage=10&sort=DATUM&sortOrder=ASC"
headers = {"accept": "application/json", "Content-Type": "application/json"}




keywords = [
    "study","studie","étude","etude","studio",
    "analysis","analyse","analisi",
    "economy","economics","wirtschaft","ökonomie","économie","economie","economia",
    "benchmark","benchmarking",
    "wirtschaftsberatung","consulting","conseil","consulenza",
    "it-dienste","dienstleistung","service","prestation","prestazione",
    "ausschreibung","appel d'offres","gara","procurement",
    "offenes verfahren","procédure ouverte","procedura aperta",
    "bundesamt für statistik","office fédéral de la statistique","bfs","seco",
    "kanton zürich","kanton luzern","region","bundesamt für bauten und logistik","bbl",
    "index","wirtschaftsforschung","tunnel"
    
]

results = []

for kw in keywords:
    data = {
        "keywords": kw,
        "stat_tm_1": "2000-01-01",
        "stat_tm_2": "2025-12-31",
        #"cpv_ob": "72000000,79300000,73100000,79311400,72314000,79416000,72320000,98300000,79310000, 79000000, 79311410"
    }
    r = requests.post(url, headers=headers, json=data)
    if r.status_code == 200:
        res = r.json().get("publication", [])
        results.extend(res)
        print(f"Fetched {len(res)} results for keyword: {kw}")
    else:
        print(f"Error {r.status_code} for keyword: {kw}")
    sleep(0.3)

# Convert to DataFrame
df = pd.DataFrame(results)
print(f"\n✅ Total collected: {len(df)}")
print(df.head())


Fetched 10 results for keyword: study
Fetched 10 results for keyword: studie
Fetched 10 results for keyword: étude
Fetched 10 results for keyword: etude
Fetched 10 results for keyword: studio
Fetched 10 results for keyword: analysis
Fetched 10 results for keyword: analyse
Fetched 10 results for keyword: analisi
Fetched 10 results for keyword: economy
Fetched 10 results for keyword: economics
Fetched 10 results for keyword: wirtschaft
Fetched 10 results for keyword: ökonomie
Fetched 10 results for keyword: économie
Fetched 10 results for keyword: economie
Fetched 10 results for keyword: economia
Fetched 10 results for keyword: benchmark
Fetched 10 results for keyword: benchmarking
Fetched 0 results for keyword: wirtschaftsberatung
Fetched 10 results for keyword: consulting
Fetched 10 results for keyword: conseil
Fetched 10 results for keyword: consulenza
Fetched 0 results for keyword: it-dienste
Fetched 10 results for keyword: dienstleistung
Fetched 10 results for keyword: service
Fetch

2- CPV search

In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from time import sleep

# ---------- PART 1: Fetch main search results ----------
url = "https://archiv.simap.ch/api/search"
headers = {"accept": "application/json", "Content-Type": "application/json"}

data = {
    "stat_tm_1": "2000-01-01",
    "stat_tm_2": "2025-12-31",
    "cpv_ob": "72000000,79300000,73100000,79311400,72314000,79416000,72320000,98300000,79310000,79000000,79311410"
}

all_results = []
page = 1

while True:
    params = {
        "pageNo": page,
        "recordsPerPage": 100,
        "sort": "DATUM",
        "sortOrder": "ASC"
    }
    r = requests.post(url, headers=headers, params=params, json=data)
    r.raise_for_status()
    publications = r.json().get("publication", [])
    if not publications:
        break

    all_results.extend(publications)
    print(f"Fetched page {page} ({len(publications)} records)")
    page += 1

df_main = pd.DataFrame(all_results)
print(f"\n✅ Total fetched: {len(df_main)} records")

# ---------- PART 2: Keep only required fields ----------
required_fields = ["id", "lang","proc", "projectid", "description", "deadline", "authName", "contType", "publicationDate"]
df_filtered = df_main[[col for col in required_fields if col in df_main.columns]].copy()

# ---------- PART 3: Fetch XML detail for each record ----------
details = []
for tender_id in tqdm(df_filtered["id"].unique(), desc="Fetching detail pages"):
    detail_url = f"https://archiv.simap.ch/api/detail?meldungsnummer={tender_id}"
    resp = requests.get(detail_url, headers={"accept": "application/xml"})
    details.append({"id": tender_id, "detail": resp.text})

df_detail = pd.DataFrame(details)

# ---------- PART 4: Merge details with main data ----------
df_final = pd.merge(df_filtered, df_detail, on="id", how="left")

# ---------- PART 5: Save ----------
df_final.to_csv("simap_tenders.csv", index=False)
print("\n💾 Saved as simap_tenders.csv")
print(df_final.head(3))


Fetched page 1 (100 records)
Fetched page 2 (100 records)
Fetched page 3 (100 records)
Fetched page 4 (100 records)
Fetched page 5 (100 records)
Fetched page 6 (100 records)
Fetched page 7 (100 records)
Fetched page 8 (100 records)
Fetched page 9 (100 records)
Fetched page 10 (100 records)
Fetched page 11 (100 records)
Fetched page 12 (100 records)
Fetched page 13 (100 records)
Fetched page 14 (100 records)
Fetched page 15 (100 records)
Fetched page 16 (100 records)
Fetched page 17 (100 records)
Fetched page 18 (100 records)
Fetched page 19 (100 records)
Fetched page 20 (100 records)
Fetched page 21 (100 records)
Fetched page 22 (100 records)
Fetched page 23 (100 records)
Fetched page 24 (100 records)
Fetched page 25 (100 records)
Fetched page 26 (100 records)
Fetched page 27 (100 records)
Fetched page 28 (100 records)
Fetched page 29 (100 records)
Fetched page 30 (100 records)
Fetched page 31 (100 records)
Fetched page 32 (100 records)
Fetched page 33 (100 records)
Fetched page 34 (10

Fetching detail pages:   3%|▎         | 363/13937 [00:17<10:40, 21.18it/s]


KeyboardInterrupt: 

df